# Real News Scraper

This notebook walks through using Beautiful Soup(python scraping module) for the first time.
Our working goal is a collection of real news texts to be used in Fake vs Real News classificaion. See https://github.com/dcurry09/FakeNews_Classifier.

We will be scraping news articles from the left, right, and center categories.  An assumption, perhaps a biased one, is that all news articles from AllSides.com are considered "real".

Output will be real news articles in a CSV file(headers include title and text only).

In [1]:
# Scraping Packages
import requests
from bs4 import BeautifulSoup
import newspaper
import textacy as tcy

# Standard Libraries
import re
from collections import defaultdict
import csv as csv
import numpy as np
import pandas as pd
import pylab as py
import operator
import time
import progressbar
import matplotlib.pyplot as plt
from collections import Counter

We define a single news article URL as a test case of scraping(single article first, then loop over nmany).

In [2]:
# Collect first Article Text
page = requests.get('https://www.allsides.com/story/warren-says-dnc-rigged-nomination')

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

Now we have an object(page) and a BS object(soup).  Let us first print out page just to see what has been defined:

In [3]:
print(page.text[:1000])

<!DOCTYPE html>
<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->
<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->
<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->
<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html  lang="en" dir="ltr" prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product# content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#"><!--<![endif]-->

<head profile="http://www.w3.org/1999/xhtml/vocab">


  <meta charset="utf-8" 

This is the same as if we had right clicked on the webpage and choosen "Inspect".  This is our HTML object.  Our job now is to identify where the article text is in the HTML and extract it.  By inspecting the web pahe HTML structure I have found a div item with a class that contains the body text.

In [4]:
body_text = soup.find_all('div', class_ = 'story-id-page-description')
print(type(body_text))
print(len(body_text))
print(body_text)

<class 'bs4.element.ResultSet'>
1
[<div class="story-id-page-description">
<p>Senator Elizabeth Warren (D-MA) answered that yes, she believed the Democratic primary was rigged to favor Hillary Clinton when asked Thursday about former Democratic National Committee Chairwoman Donna Brazile’s explosive admission that the Clinton campaign had control of the DNC before Clinton secured the nomination.</p>
</div>]


Now lets assign just the text to an object

In [5]:
for i in body_text: print(i)

<div class="story-id-page-description">
<p>Senator Elizabeth Warren (D-MA) answered that yes, she believed the Democratic primary was rigged to favor Hillary Clinton when asked Thursday about former Democratic National Committee Chairwoman Donna Brazile’s explosive admission that the Clinton campaign had control of the DNC before Clinton secured the nomination.</p>
</div>


In order to access each element in body_text we need to make it a list

In [6]:
body_text_list = body_text[0]
print(body_text_list.p)

<p>Senator Elizabeth Warren (D-MA) answered that yes, she believed the Democratic primary was rigged to favor Hillary Clinton when asked Thursday about former Democratic National Committee Chairwoman Donna Brazile’s explosive admission that the Clinton campaign had control of the DNC before Clinton secured the nomination.</p>


In [7]:
print(body_text_list.p.text)

Senator Elizabeth Warren (D-MA) answered that yes, she believed the Democratic primary was rigged to favor Hillary Clinton when asked Thursday about former Democratic National Committee Chairwoman Donna Brazile’s explosive admission that the Clinton campaign had control of the DNC before Clinton secured the nomination.


Bingo!  Ok so we have one article of text.  We need to store this first article in a CSV file, then perform many times across the website.  Lets start by creating a pandas dataframe.

In [8]:
final_text = body_text_list.p.text
test_df = pd.DataFrame({'text': [final_text]})

print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
text    1 non-null object
dtypes: object(1)
memory usage: 88.0+ bytes
None


,text
0,Senator Elizabeth Warren (D-MA) answered that ...


## Moving Beyond Single Articles
We now we expand the scope of our scraping code to include multiple websites and multiple artcicles.  We will be suing the Newspaper package of tools to help us streamline this process.  Below are a few functions that help wrap the entire workflow together(from the nice blog: https://dataflume.wordpress.com/2017/04/23/scraping-newspaper-text-and-computing-readability-statistics-in-python/)

In [9]:
def article_extractor(newspaper_url, title_topic=None):
    '''
    Extracts News article text for a given URL.
    Uses Newspaper modules and returns a pandas DF of text.
    '''
        
    dd = defaultdict(list)

    source = newspaper.build(newspaper_url, memoize_articles=False)

    print("\nExtracting articles from", newspaper_url)    
    print("# of Articles:", source.size()) 
    #print(source.articles)    
        
    arts = [i.url for i in source.articles]
    
    print("# of Article URLs:", len(arts))  
        
        
    if title_topic is None:
        relevant_arts = [i for i in arts]
    else:
        relevant_arts = [i for i in arts if title_topic in i]

    print("# of Relevant Articles:", len(relevant_arts))
        
    bar = progressbar.ProgressBar()
        
    for i in bar(relevant_arts):
        time.sleep(0.02)

        try:
            art = newspaper.build_article(i)
            art.download()
            art.parse()
            dd["title"].append(art.title)
            dd["text"].append(art.text)
        except:
            print('ERROR: URL No Longer Available...')
            continue
        
    print('DF:', pd.DataFrame.from_dict(dd).head(5))   
        
    return pd.DataFrame.from_dict(dd)

def get_articles(newspaper_url):

    results = []
    for url in newspaper_url:
        articles = article_extractor(url)
        articles["paper"] = url
        results.append(articles)
    return pd.concat(results)

def clean_text(string):

    string = re.sub(r"SIGN UP FOR OUR NEWSLETTER", "", string)
    string = re.sub(r"Read more here", "", string)
    string = re.sub(r"REUTERS", "", string)
    string = re.sub(r"\?", "'", string)
    string = re.sub(r"\n", "", string)
    return string

def preprocess_articles(articles):

    clean_arts = []
    for art in articles:
        clean_art = tcy.preprocess.preprocess_text(art,
                                          fix_unicode=True,
                                          lowercase=True,
                                          no_currency_symbols=True,
                                          no_numbers=True,
                                          no_urls=True)
        clean_arts.append(clean_art)
    return clean_arts

In [11]:
# Define the news websites to scrape from
cnn = "http://cnn.com"
allSides = "https://www.allsides.com/"
guardian = "https://www.theguardian.com/us/"
NYT = "https://www.nytimes.com/"
wapo = "https://www.washingtonpost.com/"
nbc = "https://www.nbcnews.com/"
fox = "http://www.foxnews.com/"
lat = "http://www.latimes.com/hp-2/"

site_list = [cnn, allSides, guardian, NYT, wapo, nbc, fox, lat]
#site_list = [wapo]
articles = get_articles(site_list)
articles["text"] = articles.text.map(clean_text)
articles["text"] = preprocess_articles(articles.text)

# Save to CSV
articles.to_csv('real_news.csv')

N/A% (0 of 790) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


Extracting articles from http://cnn.com
# of Articles: 790
# of Article URLs: 790
# of Relevant Articles: 790


  0% (2 of 790) |                         | Elapsed Time: 0:00:00 ETA:  0:03:48

Article `download()` failed with 404 Client Error: Not Found for url: http://www.cnn.com/hln-morning-express-tour-robin-meade on URL http://cnn.com/hln-morning-express-tour-robin-meade
ERROR: URL No Longer Available...


 19% (157 of 790) |####                   | Elapsed Time: 0:02:31 ETA:  0:07:20/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 33434
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 33437
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 36868
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/lib/python3.6/site-packages/PIL/TiffImag

...

You must `download()` an article first!
ERROR: URL No Longer Available...


 98% (69 of 70) |######################## | Elapsed Time: 0:01:11 ETA:  0:00:02

You must `download()` an article first!
ERROR: URL No Longer Available...


100% (70 of 70) |#########################| Elapsed Time: 0:01:18 Time: 0:01:18


You must `download()` an article first!
ERROR: URL No Longer Available...
DF:                                                 text  \
0  New York City police investigators say a 2010 ...   
1  House Republicans produced an ambitious propos...   
2  In a highly unusual move, the Justice Departme...   
3  A massive U.S. report concludes that the evide...   
4  Sukhrob Sobirov was 20 when he left Uzbekistan...   

                                               title  
0  NYPD gathering evidence to arrest Harvey Weins...  
1  House Republicans produced an ambitious tax ov...  
2  Trump administration asks Supreme Court to pun...  
3  U.S. report contradicts Trump team: Warming is...  
4  Uzbek community in New York wary of being tied...  
